In [1]:
import requests ##Peticiones
from bs4 import BeautifulSoup ## Scrapping
import pandas as pd  ##Dataframe


### Función auxiliar para crear las diferentes conexiones al necesitar navegar por la página.
### Reibe como parametro la url hacía donde va la conexión y se concatena a la URL base.

In [2]:
def abrir_conexion(url):
    url_base="https://www.ubereats.com" ##base
    headers = {'Accept': 'text/html'} ##Encabezado para que acepte la conexión
    url_final=url_base+url ##concatenar direcciones
    pagina=requests.get(url_final,headers=headers) ##Hacer el request a la pag
    if pagina.status_code ==200: ##Control para la conexión
        print("Conexión establecida")
        contenido=BeautifulSoup(pagina.content,"html.parser") ##Bajar html
        return contenido
    else:
        print("Conexión fallido")
        return False
    



### Llamado, apertura y consecución del contenido de la página:

* Importante, si se desea acceder a la información de otros paises, basta con cambiar '/co/location', por 'xx/location', donde xx representa el dominio de cualquier país. Ejemplo: 'ec/location', para obtener información de Ecuador.

### Extracción de las diferentes ciudades que tiene UberEats registradas en Colombia (o país deseado), además de la extracción de las urls para navegar allí :

* Se extraen todas las ciudades y las respectivas URL hacía donde dirigen. Esto con el fin de que sea escalable, por si el día de mañana se agregan otras ciudades no presentes en el momento.

In [3]:
divTag=[] #Almacen etiquetas
while(len(divTag)==0): ##Asegurar extracción de contenido
    url = '/co/location' ##Url de Colombia. #Podría ser de otro país
    contenido=abrir_conexion(url) ##abrir pagina 
    divTag = contenido.find_all("div", {"class": "db dc dd de df dg dh"}) ##Etiqueta donde están almacenados las ciudades del país
ciudades=divTag[0].find_all("span", {"class": "bj bk c3 au"}) ##Obtener las ciudades del país seleccionado
url_ciudades=divTag[0].find_all("a") ##Obtener las urls que dirigen

Conexión establecida


In [4]:
ciudades=[ciudad.text for ciudad in ciudades] ## Almacenar la lista de ciudades que tiene registradas UberEats en Colombia/ país seleccionado
url_ciudades=[url["href"] for url in url_ciudades] ##Almacenar Urls para cada ciudad

In [5]:
print(ciudades)
print(url_ciudades)

['Ambato', 'Cuenca', 'Guayaquil', 'Quito']
['/ec/location/ambato', '/ec/location/cuenca', '/ec/location/guayaquil', '/ec/location/quito']


In [6]:
ciudad_categoria={} ##Guardar las ciudades y las categorias de cada ciudad
ciudad_categoria_url={} ##Guardar URL

### Tratando de navegar hacía las Urls de las diferentes ciudades Colombianas o del país seleccionado, almacenar las diferentes categorias que posee. 
* Fue necesario usar el ciclo while, ya que cuando se hacían multiples request seguidos, muchas veces retornaba vacios. Con el while se asegura que repita la petición hasta que obtenga un resultado.

In [7]:
for url in range(0,len(url_ciudades)): ##Iterar sobre las ciudades
    categorias_etiqueta=[] ##auxiliar control
    while len(categorias_etiqueta)==0: ##Control para respuestas vacias por recurrencias
        #print(url_ciudades[url]) debuggin
        contenido_ciudad=abrir_conexion(url_ciudades[url]) ##abrir conexión
        while(contenido_ciudad==False):
                    contenido_ciudad=abrir_conexion(url_ciudades[url]) ##abrir conexión


        categorias_etiqueta=contenido_ciudad.find_all("a", {"class": "ey d8 ez"}) ##Obtener las categorias
        categorias_etiqueta2=contenido_ciudad.find_all("a", {"class": "eq d8 er"}) ##Obtener categorias (alternas)
        categorias_etiqueta+=categorias_etiqueta2 #Unirlas
        

    categorias_ciudad=[etiqueta.text for etiqueta in categorias_etiqueta] ##Nombres de categoria por ciudad
    url_categorias=[etiqueta["href"] for etiqueta in categorias_etiqueta] ## Obtener Url de las caterogias por ciudad
    ciudad_categoria[ciudades[url]]=categorias_ciudad ## Guardar ciudad y respectivas categorías.
    ciudad_categoria_url[ciudades[url]]=url_categorias ## Guardar ciudad y respectivas categorías.



/ec/location/ambato
Conexión establecida
/ec/location/cuenca
Conexión establecida
/ec/location/guayaquil
Conexión establecida
/ec/location/quito
Conexión fallido
Conexión fallido
Conexión establecida


### Abrir las conexiones con cada categoria por ciudad y extraer los restaurantes. PARTE CRÍTICA:

In [9]:
ciudad_categoria_definitiva={}  ##Almacenes de datos y urls.
control_url=[]

In [15]:
for i in ciudad_categoria_url: #Iterar sobre las diferentes ciudadees- categorías
    
    for url in ciudad_categoria_url[i]: ##iterar en URL
        
        sitio_etiqueta=[]
        while len(sitio_etiqueta)==0: ##Control de concurrencia
            contenido_categoria=abrir_conexion(url) ##abrir conexión
            while(contenido_categoria==False): ##Control concurrencia
                contenido_categoria=abrir_conexion(url) ##abrir conexión


            sitio_etiqueta=contenido_categoria.find_all("h3",{"class":"ek bk bl au el al aj em dk"}) 
                             ##Obtener las categorias
        url_restaurante=[]
        for restaurante in sitio_etiqueta:
            url_restaurante.append(restaurante.find_all("a")[0]["href"]) ##Extraer url del restaurante

        ciudad_categoria_definitiva[str(i)+"-"+str(url)]=url_restaurante ##Almacenando la lista de url de restaurantes por ciudad-categoría
        




Conexión fallido
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión fallido
Conexión establecida
Conexión establecida
Conexión fallido
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión fallido
Conexión fallido
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión fallido
Conexión establecida
Conexión fallido
Conexión fallido
Conexión fallido
Conexión establecida
Conexión establecida
Conexión fallido
Conexión establecida
Conexión establecida
Conexión fallido
Conexión establecida
Conexión fallido
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión fallido
Conexión establecida
Conexión establecida
Conexión fallido
Conexión establecida
Conexión establecida
Conexión establecida
Conexión fallido
Conexión establecida
Conexión establecida
Conexión e

In [16]:

listarestaurantes=[] ##Almacenar todos los restaurantes. 

### Se accede a cada uno de los diferentes restaurantes, para extraer los atributos requeridos. Posterior a esto se guardan todos como diccionarios, para finalmente llevarlos a un dataFrame.

* El principal problema que se tuvo acá, fue el hecho de que muchos restaurantes varían en su estrcutura HTML, según el estado en el que se encuentren en el momento. Si está abierto, si está cerrado, si es exclusivo de uberEats, si es nuevo, si no está disponible. Estos factores alteran la estructura y fue necesario programar una extracción más dinámica, con varias casuisticas:

In [17]:
#cont=0 debugging
for ciudad_categoriaaux in ciudad_categoria_definitiva:
    for restaurante in ciudad_categoria_definitiva[ciudad_categoriaaux]:
        nombre=[]
        categoria_especifica=[]
        calificacion=[]
        reviews=[]
        direccion=[]
        
        #cont+=1 debugging
        #print("Conteo: ", cont) debugging
        

        ##ABIERTO Y CERRADO: ###WHILE VALIDA RESPUESTA
        if restaurante in control_url: ##Validador de repetición
            #print("LO TENGO") ###debugging
            pass 
        else:
            control_url.append(restaurante)
            contenido_restaura=abrir_conexion(restaurante) ## Abrir conexión restaurante
            while(contenido_restaura==False):## concurrencia
                contenido_restaura=abrir_conexion(restaurante) ## Abrir conexión restaurante

            contenido_restaura_aux=contenido_restaura.find("div",{"class":"au at dc dd de df af aq dg"}) #estructura 1
            if contenido_restaura_aux==None:
                
                contenido_restaura_aux=contenido_restaura.find("div",{"class":"au at d8 d9 da db af aq dc"}) #estructura 2
                if contenido_restaura_aux==None:
                    pass 
                    
            nombre=contenido_restaura_aux.find_all("h1",{"class":"de df dg as"}) #Obtener el nombre, tipo 1
            if len(nombre)==0:
                nombre=contenido_restaura_aux.find_all("h1",{"class":"di dj dk as"}) ##Nombre, alternativo 2
                if len(nombre)==0:  
                    nombre=contenido_restaura_aux.find_all("h1",{"class":"d4 d5 d6 au"}) ##Nombre, alternativo 3
                    if len(nombre)==0:
                        nombre=""##Peor de los casos, no está el nombre

            categoria_especifica=contenido_restaura_aux.find_all("div",{"class":"c6 cv ba as dh"})  ##Categoria tipo 1
            if len(categoria_especifica)==0:
                categoria_especifica=contenido_restaura_aux.find_all("div",{"class":"c6 cv ba as dl"}) ## Categoria, alternativa 
                if len(categoria_especifica)==0:
                    categoria_especifica=contenido_restaura_aux.find_all("div",{"class":"ct cu ck au ft"}) #tipo 3
                    if len(categoria_especifica)==0:

                        categoria_especifica="" ##Peor de los casos, no hay categoría

            calificacion=contenido_restaura_aux.find_all(lambda x: x.name == 'div' and x.get('class') == ['c4']) ##Calificación 1

            if len(calificacion)==0: 
                calificacion=contenido_restaura_aux.find_all(lambda x: x.name == 'div' and x.get('class') == ['au']) ## tipo 2
                if len(calificacion)==0:
                    calificacion=contenido_restaura_aux.find_all(lambda x: x.name == 'div' and x.get('class') == ['as']) ##tipo 3
                    if len(calificacion)==0:
                        calificacion=contenido_restaura_aux.find_all(lambda x: x.name == 'div' and x.get('class') == ['an']) ##tipo 4
                        if(len(calificacion))==0:
                            calificacion=""##Peor de los casos, calificación vacía






            reviews=contenido_restaura_aux.find_all(lambda x: x.name == 'div' and x.get('class') == ['dj']) ##Revies tipo 1

            if len(reviews)==0:
                reviews=contenido_restaura_aux.find_all(lambda x: x.name == 'div' and x.get('class') == ['dn']) ##Reviews alternativo
                if len(reviews)==0:
                    reviews=contenido_restaura_aux.find_all(lambda x: x.name == 'div' and x.get('class') == ['du']) ##Reviews alternativo
                    if len(reviews)==0:

                        reviews="" ##Peor de los casos, reviews está vacío 

            direccion=contenido_restaura_aux.find_all("p",{"class":"c6 cv ba as dm"}) ##Dirección tiene 5 tipos de variaciones, tipo 1  
            if len(direccion)==0:
                direccion=contenido_restaura_aux.find_all("p",{"class":"c6 cv ba as dq"}) ##tipo 2
                if len(direccion)==0:
                    direccion=contenido_restaura_aux.find_all("p",{"class":"c6 cv ba as dl"}) ## tipo 3
                    if len(direccion)==0:
                        direccion=contenido_restaura_aux.find_all("p",{"class":"c6 cv ba as dp"}) ##tipo 4
                        if len(direccion)==0:
                            direccion=contenido_restaura_aux.find_all("p",{"class":"ct cu ck au h8"}) ##tipo 5
                            if len(direccion)==0:

                                direccion=""  ##Peor de los casos, está vacía
            if (nombre != ""): ##Extraer texto de cada etiqueta:

                nombre=nombre[0].text
                
            if (categoria_especifica != ""):
                categoria_especifica=categoria_especifica[0].text
                
            if (calificacion != ""):
                try:
                    
                    calificacion=calificacion[0].text

                except:
                    
                    calificacion=""
            
            if (reviews != ""):
                reviews=reviews[0].text
                
            if (direccion != ""):
                direccion=direccion[0].text
                
            #print("-------------------------||||||||||||||||||||||||||||||-------------------------")
            ###DEBUGGING:
    #print(nombre[0].text)
    #print(categoria_especifica[0].text)
    #print(calificacion[0].text)
    #print(reviews[0].text)
    #print(direccion[0].text)
        #print(nombre)
        #print(categoria_especifica)
        #print(calificacion)
        #print(reviews)
        #print(direccion)
        #print(nombre[0].text)
        #print(categoria_especifica[0].text)
        #print(calificacion[0].text)
        #print(reviews[0].text)
        #print(direccion[0].text)

            restaux={"Nombre":nombre,"Categoria_Especifica":categoria_especifica,"Calificacion":calificacion,"Reviews":reviews,"Direccion":direccion, "Url":"https://www.ubereats.com"+restaurante} ##Llenar diccionario de información
        listarestaurantes.append(restaux) #Añadir a la lista de diccionarios
        #print(restaux) debugging

    

 'Direccion': 'Nuñez De Vela E3-13, Quito 170135, Ecuador, Local 1 , Pichincha 170135\xa0•\xa0Más información', 'Url': 'https://www.ubereats.com/ec/quito/food-delivery/corvinas-don-jimmy/CImBjB41SmW5SmgzceoeNQ'}
Conteo:  2386
Conexión establecida
-------------------------||||||||||||||||||||||||||||||-------------------------
{'Nombre': 'Cevicheria Pepe 3', 'Categoria_Especifica': '$$ • Mariscos • Latina • Opciones para alérgicos', 'Calificacion': '4.5', 'Reviews': '(23)', 'Direccion': 'Av. Atahualpa 171, Quito 170135, Ecuador, Piso 1, Pichincha 170135\xa0•\xa0Más información', 'Url': 'https://www.ubereats.com/ec/quito/food-delivery/cevicheria-pepe-3/1GL2dHmRR5WcKoP_QLsTwg'}
Conteo:  2387
Conexión fallido
Conexión establecida
-------------------------||||||||||||||||||||||||||||||-------------------------
{'Nombre': 'Cevicheria Pepe 3 (Plaza Foch)', 'Categoria_Especifica': '$$ • Mariscos • Latina', 'Calificacion': '', 'Reviews': '', 'Direccion': 'Juan Rodríguez E7-22, Quito, Pichincha 

KeyboardInterrupt: 

In [18]:
restaurantes=pd.DataFrame(listarestaurantes) ##Volver Data frame
restaurantes.to_csv('restaurantes_09062020pm.csv', encoding='utf-8') ## Exportar a csv, para luego limpiar con el segundo notebook, especificar nombre relacionado con el país